In [3]:
%run Utils.ipynb

In [4]:
class Verifier_first_stage():
    
    def __init__(self):
        """
        """
    
    def verify(self, proof_stage_one: dict, domian_size: int):
            index = self.get_index(proof_stage_one=proof_stage_one, domian_size=domian_size)
            self.verifiy_path_and_values(proof_stage_one=proof_stage_one, index=index, domian_size=domian_size)
            coeffs = self.get_coefficients(proof_stage_one=proof_stage_one)
            self.validate_cp_value_in_index(proof_stage_one=proof_stage_one, coeffs=coeffs)

    def verifiy_path_and_values(self, proof_stage_one: dict, index: int, domian_size: int):
        names = ['p1', 'p2', 'p3', 'cp']
        for name in names:
            internal_proof_dict = proof_stage_one[name]
            root = internal_proof_dict['root']
            value_at_index = internal_proof_dict['value']
            path = internal_proof_dict['path']
#             self.verify_path_by_index(
#                 root=root, expected_value=value_at_index, index=index, domain_size=domian_size, path=path
#             )
            verify_path_by_index(root=root, expected_value=value_at_index, path=path)
            
    def get_index(self, proof_stage_one: dict, domian_size: int):
        #print(f"{proof_stage_one['cp']=}")
        cp_tree_root = proof_stage_one['cp']['root']
        return (fiat_shamir_random(cp_tree_root)%domian_size)
    
    def get_coefficients(self, proof_stage_one:dict) -> dict:
        names = ['p1', 'p2', 'p3']
        return {name:fiat_shamir_random(proof_stage_one[name]['root']) for name in names}
    
    def validate_cp_value_in_index(self, proof_stage_one: dict, coeffs: dict):
        names = ['p1', 'p2', 'p3']
        expected_value = 0
        for name in names:
            expected_value_int = ((proof_stage_one[name]['value'])[1]*coeffs[name])
            expected_value += expected_value_int%PRIME            
        assert proof_stage_one['cp']['value'][1] == expected_value
    
#     def verify_path_by_index(self, root: str, expected_value: tuple, index: int, domain_size: int, path: dict):

#         index_size = int(log(domain_size, 2))
#         index_as_str = format(index, f'#0{index_size+2}b')
#         # shift from '0b1110' to '1110'
#         index_as_str = index_as_str[2:] 
#         key=root
#         while(index_as_str):
#             value = path[key]

#             #Verify the hash:
#             assert hash_tow_elements(value[0], value[1]) == key

#             direction_bit = int(index_as_str[0])
#             key = value[direction_bit]
#             index_as_str = index_as_str[1:]

#         #Now the key is the hash of the required index. Reauired value = tree[key] = (coset, CP[coset])
#         value = path[key]
#         assert hash_one_elements(value) == key
#         assert value == expected_value

In [6]:
def verify_second_stage(proof_stage_two: dict, root_stage_one: str, cp_degree: int):
    #proof structure: {stage: {'root': merkle_root, 'pathes':[pathes],'random':random number for naxt-stage}}
    #path in pathes structure: ((point),{path})

    for stage,proof in proof_stage_two.items():
        last_stage = stage
        
        #Verify pathes:
        current_stage_point = []
        for path in proof['pathes']:
            verify_path_by_index(proof['root'], expected_value = path[0], path= path[1])
            if path[0] not in current_stage_point:
                current_stage_point.append(path[0])
                
        #Verify random:
        assert proof['random'] == fiat_shamir_random(proof['root'])
        next_stage_points = []
        
        #compute next stage points for the first stage:
        if stage == 0:
            #Verify that the original function of stage two is cp from stage on)
            assert proof['root'] == root_stage_one
            for i in range(0,len(current_stage_point),2):
                a,f_a = current_stage_point[i]
                neg_a, neg_f_a = current_stage_point[i+1]
                assert neg_a**2== a**2 #Verify that two conssecutive values are inverse elements of each other
                y = f_a + (f_a - neg_f_a)/(2*a)*(proof['random']-a)
                next_stage_points.append((a**2,y))
                
        #compute next stage points in the general case:
        else:
            for point_a, point_neg_a in zip (current_stage_point, calculated_next_stage_points):
                a,f_a = point_a
                neg_a, neg_f_a = point_neg_a
                assert neg_a**2== a**2 #Verify that two conssecutive values are inverse elements of each other
                y = f_a + (f_a - neg_f_a)/(2*a)*(proof['random']-a)
                next_stage_points.append((a**2,y))
                
        calculated_next_stage_points = next_stage_points
    
    #veify stages number:
    assert last_stage == log(cp_degree,2)-1
    
    #veify last FRI stage is constant:
    for i in range(len(calculated_next_stage_points)-1):
        assert calculated_next_stage_points[i][1] == calculated_next_stage_points[i+1][1]
        
    return True